# Bilingual Generalization Experiment

In [ ]:
import os
import sys
sys.path.append('../talk_tuner/')
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm.auto import tqdm

from collections import OrderedDict

from src.dataset import llama_v2_prompt

import numpy as np

import torch
from src.probes import LinearProbeClassification

from src.intervention_utils import return_classifier_dict



device = "cuda"
torch_device = "cuda"

## Instantiate the model

In [ ]:
access_token = ''

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf", token=access_token, padding_side='left')
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-13b-chat-hf", token=access_token)
model.half().cuda();
model.eval();

## Load the probe weights here

In [ ]:
classifier_type = LinearProbeClassification
classifier_directory = "../probe_checkpoints/orig_controlling_probe"
return_user_msg_last_act = True
include_inst = True
layer_num = None
mix_scaler = False
residual_stream = True
logistic = True
sklearn = False

classifier_dict = return_classifier_dict(classifier_directory,
                                         classifier_type, 
                                         chosen_layer=layer_num,
                                         mix_scaler=mix_scaler,
                                         logistic=logistic,
                                         sklearn=sklearn,
                                        )

## Hyperparameters

In [ ]:
normalized=False
# Sampling hyperparameters
generation_temperature = 0
generation_top_p = 1

N = 8 # Intervention Strength

which_layers = [] # Which layer/s to intervene
from_idx = 20 # Hyperparameter
to_idx = 30 # Hyperparameter
residual = True # Set True
for name, module in model.named_modules():
    if residual and name!= "" and name[-1].isdigit():
        layer_num = name[name.rfind("model.layers.") + len("model.layers."):]
        if from_idx <= int(layer_num) < to_idx:
            which_layers.append(name)
    elif (not residual) and name.endswith(".mlp"):
        layer_num = name[name.rfind("model.layers.") + len("model.layers."):name.rfind(".mlp")]
        if from_idx <= int(layer_num) < to_idx:
            which_layers.append(name)
modified_layer_names = which_layers
        
attribute = "age" # which attribute to intervene

## Batched Intervention code

From the TalkTuner codebase

In [30]:
from baukit import TraceDict
from torch import nn


if '<pad>' not in tokenizer.get_vocab():
    tokenizer.add_special_tokens({"pad_token":"<pad>"})

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id
assert model.config.pad_token_id == tokenizer.pad_token_id, "The model's pad token ID does not match the tokenizer's pad token ID!"

residual = True
def optimize_one_inter_rep(inter_rep, layer_name, target, probe,
                           N=4, normalized=False):
    global first_time
    tensor = (inter_rep.clone()).to(torch_device).requires_grad_(True)
    rep_f = lambda: tensor
    target_clone = target.clone().to(torch_device).to(torch.float)

    cur_input_tensor = rep_f().clone().detach()


    if normalized:
        cur_input_tensor = rep_f() + target_clone.view(1, -1) @ probe.proj[0].weight * N * 100 / rep_f().norm() 
    else:

        # print(f"rep_f().shape: {rep_f().shape}")
        # print(f"target_clone.view(1, -1).shape: {target_clone.view(1, -1).shape}")
        # print(f"N: {N}")
        # print(f"multiplication: {( target_clone.view(1, -1) @ probe.proj[0].weight ).shape}")

        cur_input_tensor = rep_f() + target_clone.view(1, -1) @ probe.proj[0].weight * N
    return cur_input_tensor.clone()


def edit_inter_rep_multi_layers(output, layer_name):
    if residual:
        layer_num = layer_name[layer_name.rfind("model.layers.") + len("model.layers."):]
    else:
        layer_num = layer_name[layer_name.rfind("model.layers.") + len("model.layers."):layer_name.rfind(".mlp")]
    layer_num = int(layer_num)
    probe = classifier_dict[attribute][layer_num + 1]
    print(f"output: {output.shape}")
    print(f"output[0][:,-1].shape: {output[:, -1, :].shape}")
    cloned_inter_rep = output[:, -1,:].detach().clone().to(torch.float)
    print(f"cloned_inter_rep: {cloned_inter_rep.shape}")
    with torch.enable_grad():
        cloned_inter_rep = optimize_one_inter_rep(cloned_inter_rep, layer_name, 
                                                  cf_target, probe,
                                                  N=N,
                                                  normalized=False)
    output[:,  -1,:] = cloned_inter_rep.to(torch.float16)
    return output


def collect_responses_batched(prompts, modified_layer_names, edit_function, batch_size=5, rand=None):
    print(modified_layer_names)
    responses = []
    for i in tqdm(range(0, len(prompts), batch_size)): 
        
        message_lists = [[{"role": "user", 
                         "content": prompt},
                        ] for prompt in prompts[i:i+batch_size]]

        # Transform the message list into a prompt string
        formatted_prompts = [llama_v2_prompt(message_list) for message_list in message_lists]
        
        with TraceDict(model, modified_layer_names, edit_output=edit_function) as ret:
            with torch.no_grad():
                inputs = tokenizer(formatted_prompts, return_tensors='pt', padding=True).to('cuda')
                tokens = model.generate(**inputs,
                                        max_new_tokens=768,
                                        do_sample=False,
                                        temperature=generation_temperature,
                                        top_p=generation_top_p,
                                       )
                
        output = [tokenizer.decode(seq, skip_special_tokens=True).split('[/INST]')[1] for seq in tokens]
        responses.extend(output)

    return responses

## English Inference

In [34]:
cf_target = [0, 0, 0, 0]
# cf_target = [0, 0]

cf_target[3] = 1
cf_target = torch.Tensor([cf_target])

# and we want the strength to be 8
N = 8

# We want to modify layers 20 to 30
modified_layer_names = which_layers

batch_size = 2
# and we have an array of questions
questions = ["Can you suggest five fun outdoor activities for me?", 
             "How can I learn to use technology more efficiently?",]
results = collect_responses_batched(questions, modified_layer_names, edit_inter_rep_multi_layers, batch_size=batch_size, rand=None)

['model.layers.20', 'model.layers.21', 'model.layers.22', 'model.layers.23', 'model.layers.24', 'model.layers.25', 'model.layers.26', 'model.layers.27', 'model.layers.28', 'model.layers.29']


  0%|          | 0/1 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


output: torch.Size([2, 119, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 119, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 119, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 119, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 119, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 119, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 119, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 119, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 119, 5120])
outpu

100%|██████████| 1/1 [00:18<00:00, 18.67s/it]

output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([2, 5120])
cloned_inter_rep: torch.Size([2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: 

In [35]:
for i in range(len(questions)):
    text = f"USER: {questions[i]}\n\n"
    text += "-" * 50 + "\n"
    text += f"Intervened:\n"
    text += f"CHATBOT: {results[i]}"
    text += "\n\n" + "-" * 50 + "\n"
    
    print(text)

USER: Can you suggest five fun outdoor activities for me?

--------------------------------------------------
Intervened:
CHATBOT:   Of course! I'd be happy to help you find some fun outdoor activities. Here are five suggestions:

1. Hiking or walking: Take a leisurely walk or hike through a nearby park or nature reserve to enjoy the fresh air and scenery. You can choose a route that suits your fitness level and enjoy the peaceful surroundings.
2. Picnic: Pack a basket with your favorite food and drinks and enjoy a picnic in a park or by a body of water. You can also bring along a blanket and some comfortable cushions to make the experience even more enjoyable.
3. Cycling: Dust off your bicycle and take a leisurely ride through a scenic route. You can explore local parks or bike trails and enjoy the exercise and fresh air.
4. Birdwatching: Take a pair of binoculars and a notebook and go birdwatching in a nearby park or nature reserve. You can identify different species of birds and lea

## Spanish Inference

In [36]:
sys.path.append('../')
from translator import ConversationTranslator


spanish_questions = [ConversationTranslator('es').translate_text(question) for question in questions]
print(spanish_questions)
responses = []
for i in tqdm(range(0, len(spanish_questions), batch_size)): 
  
  message_lists = [[{"role": "user", 
                  "content": prompt},
                  ] for prompt in spanish_questions[i:i+batch_size]]

  # Transform the message list into a prompt string
  formatted_prompts = [llama_v2_prompt(message_list) for message_list in message_lists]
  
  with torch.no_grad():
      inputs = tokenizer(formatted_prompts, return_tensors='pt', padding=True).to('cuda')
      tokens = model.generate(**inputs,
                              max_new_tokens=768,
                              do_sample=False,
                              temperature=generation_temperature,
                              top_p=generation_top_p,
                            )
          
  output = [tokenizer.decode(seq, skip_special_tokens=True).split('[/INST]')[1] for seq in tokens]

  responses.extend(output)





['¿Puedes sugerirme cinco actividades al aire libre divertidas?', '¿Cómo puedo aprender a usar la tecnología de manera más eficiente?']


100%|██████████| 1/1 [00:22<00:00, 22.46s/it]


In [37]:
for i in range(len(spanish_questions)):
    text = f"USER: {spanish_questions[i]}\n\n"
    text += "-" * 50 + "\n"
    text += f"Intervened:\n"
    text += f"CHATBOT: {responses[i]}"
    text += "\n\n" + "-" * 50 + "\n"
    
    print(text)

USER: ¿Puedes sugerirme cinco actividades al aire libre divertidas?

--------------------------------------------------
Intervened:
CHATBOT:   ¡Por supuesto! Aquí te sugiero cinco actividades al aire libre divertidas y seguras:

1. Caminar o hacer senderismo: Elige un parque o un camino en la naturaleza y disfruta del aire fresco y la belleza del entorno. Puedes hacer un recorrido corto o largo, según tus preferencias y capacidad.
2. Practicar yoga al aire libre: Busca un lugar tranquilo y soleado, como un parque o una playa, y establece tu propio espacio de yoga. El aire libre te brindará una sensación de liberación y conexión con la naturaleza.
3. Hacer un picnic: Prepara un delicioso picnic con tus seres queridos en un parque o en un lugar tranquilo. Disfruta de la comida y la compañía en un entorno natural.
4. Realizar actividades creativas al aire libre: Si te gusta dibujar, pintar o hacer fotos, busca un lugar con buena luz natural y comienza a crear. El aire libre te inspirará y

## load the bilingual probe

In [14]:
classifier_type = LinearProbeClassification
classifier_directory = "../probe_checkpoints/bilingual_controlling_probe"
return_user_msg_last_act = True
include_inst = True
layer_num = None
mix_scaler = False
residual_stream = True
logistic = True
sklearn = False

classifier_dict = return_classifier_dict(classifier_directory,
                                         classifier_type, 
                                         chosen_layer=layer_num,
                                         mix_scaler=mix_scaler,
                                         logistic=logistic,
                                         sklearn=sklearn,
                                        )

In [15]:
normalized=False
# Sampling hyperparameters
generation_temperature = 0
generation_top_p = 1

N = 8 # Intervention Strength

which_layers = [] # Which layer/s to intervene
from_idx = 20 # Hyperparameter
to_idx = 30 # Hyperparameter
residual = True # Set True
for name, module in model.named_modules():
    if residual and name!= "" and name[-1].isdigit():
        layer_num = name[name.rfind("model.layers.") + len("model.layers."):]
        if from_idx <= int(layer_num) < to_idx:
            which_layers.append(name)
    elif (not residual) and name.endswith(".mlp"):
        layer_num = name[name.rfind("model.layers.") + len("model.layers."):name.rfind(".mlp")]
        if from_idx <= int(layer_num) < to_idx:
            which_layers.append(name)
modified_layer_names = which_layers
        
attribute = "age" # which attribute to intervene

In [16]:
from baukit import TraceDict
from torch import nn


if '<pad>' not in tokenizer.get_vocab():
    tokenizer.add_special_tokens({"pad_token":"<pad>"})

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id
assert model.config.pad_token_id == tokenizer.pad_token_id, "The model's pad token ID does not match the tokenizer's pad token ID!"

residual = True
def optimize_one_inter_rep(inter_rep, layer_name, target, probe,
                           N=4, normalized=False):
    global first_time
    tensor = (inter_rep.clone()).to(torch_device).requires_grad_(True)
    rep_f = lambda: tensor
    target_clone = target.clone().to(torch_device).to(torch.float)

    cur_input_tensor = rep_f().clone().detach()


    if normalized:
        cur_input_tensor = rep_f() + target_clone.view(1, -1) @ probe.proj[0].weight * N * 100 / rep_f().norm() 
    else:

        # print(f"rep_f().shape: {rep_f().shape}")
        # print(f"target_clone.view(1, -1).shape: {target_clone.view(1, -1).shape}")
        # print(f"N: {N}")
        # print(f"multiplication: {( target_clone.view(1, -1) @ probe.proj[0].weight ).shape}")

        cur_input_tensor = rep_f() + target_clone.view(1, -1) @ probe.proj[0].weight * N
    return cur_input_tensor.clone()


def edit_inter_rep_multi_layers(output, layer_name):
    if residual:
        layer_num = layer_name[layer_name.rfind("model.layers.") + len("model.layers."):]
    else:
        layer_num = layer_name[layer_name.rfind("model.layers.") + len("model.layers."):layer_name.rfind(".mlp")]
    layer_num = int(layer_num)
    probe = classifier_dict[attribute][layer_num + 1]
    print(f"output: {output.shape}")
    print(f"output[0][:,-1].shape: {output[0][:,-1].shape}")
    cloned_inter_rep = output[:, -1,:].unsqueeze(0).detach().clone().to(torch.float)
    print(f"cloned_inter_rep: {cloned_inter_rep.shape}")
    with torch.enable_grad():
        cloned_inter_rep = optimize_one_inter_rep(cloned_inter_rep, layer_name, 
                                                  cf_target, probe,
                                                  N=N,
                                                  normalized=False)
    output[:,  -1,:] = cloned_inter_rep.to(torch.float16)
    return output


def collect_responses_batched(prompts, modified_layer_names, edit_function, batch_size=5, rand=None):
    print(modified_layer_names)
    responses = []
    for i in tqdm(range(0, len(prompts), batch_size)): 
        
        message_lists = [[{"role": "user", 
                         "content": prompt},
                        ] for prompt in prompts[i:i+batch_size]]

        # Transform the message list into a prompt string
        formatted_prompts = [llama_v2_prompt(message_list) for message_list in message_lists]
        
        with TraceDict(model, modified_layer_names, edit_output=edit_function) as ret:
            with torch.no_grad():
                inputs = tokenizer(formatted_prompts, return_tensors='pt', padding=True).to('cuda')
                tokens = model.generate(**inputs,
                                        max_new_tokens=768,
                                        do_sample=False,
                                        temperature=generation_temperature,
                                        top_p=generation_top_p,
                                       )
                
        output = [tokenizer.decode(seq, skip_special_tokens=True).split('[/INST]')[1] for seq in tokens]
        responses.extend(output)

    return responses

In [17]:
cf_target = [0, 0, 0, 0]
# cf_target = [0, 0]

cf_target[0] = 1
cf_target = torch.Tensor([cf_target])

# and we want the strength to be 8
N = 8

# We want to modify layers 20 to 30
modified_layer_names = which_layers

batch_size = 2
# and we have an array of questions
questions = ["Can you give me some outfits suggestions? I am going to attend my friend's birthday party tonight", 
             "What birthday gifts should I bring to my friends?",]
results = collect_responses_batched(questions, modified_layer_names, edit_inter_rep_multi_layers, batch_size=batch_size, rand=None)

['model.layers.20', 'model.layers.21', 'model.layers.22', 'model.layers.23', 'model.layers.24', 'model.layers.25', 'model.layers.26', 'model.layers.27', 'model.layers.28', 'model.layers.29']


  0%|          | 0/1 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


output: torch.Size([2, 132, 5120])
output[0][:,-1].shape: torch.Size([132])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 132, 5120])
output[0][:,-1].shape: torch.Size([132])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 132, 5120])
output[0][:,-1].shape: torch.Size([132])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 132, 5120])
output[0][:,-1].shape: torch.Size([132])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 132, 5120])
output[0][:,-1].shape: torch.Size([132])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 132, 5120])
output[0][:,-1].shape: torch.Size([132])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 132, 5120])
output[0][:,-1].shape: torch.Size([132])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 132, 5120])
output[0][:,-1].shape: torch.Size([132])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 132, 5120])
output[0][:,-

100%|██████████| 1/1 [00:11<00:00, 11.11s/it]

output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([1])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([1])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([1])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([1])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([1])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([1])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([1])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([1])
cloned_inter_rep: torch.Size([1, 2, 5120])
output: torch.Size([2, 1, 5120])
output[0][:,-1].shape: torch.Size([1])
cloned_i

In [18]:
for i in range(len(questions)):
    text = f"USER: {questions[i]}\n\n"
    text += "-" * 50 + "\n"
    text += f"Intervened:\n"
    text += f"CHATBOT: {results[i]}"
    text += "\n\n" + "-" * 50 + "\n"
    
    print(text)

USER: Can you give me some outfits suggestions? I am going to attend my friend's birthday party tonight

--------------------------------------------------
Intervened:
CHATBOT:   Of course! I'd be happy to help you find some outfit ideas for your friend's birthday party tonight. Before I start suggesting outfits, can you tell me a bit more about the party? What is the theme, if any, and what is the dress code? Also, what is your personal style like? Do you like dressing up in fancy clothes or do you prefer something more casual and playful? Knowing these details will help me give you more tailored suggestions. 😊

--------------------------------------------------

USER: What birthday gifts should I bring to my friends?

--------------------------------------------------
Intervened:
CHATBOT:   Hello! I'd be happy to help you find some birthday gift ideas for your friends! However, I want to point out that it's important to consider the individual interests and preferences of each friend

In [ ]:
sys.path.append('../')
from translator import ConversationTranslator


questions = ["Can you give me some outfits suggestions? I am going to attend my friend's birthday party tonight.", 
             "What birthday gifts should I bring to my friends? ",]

spanish_questions = [ConversationTranslator('es').translate_text(question) for question in questions]
print(spanish_questions)
responses = []

for i in tqdm(range(0, len(spanish_questions), batch_size)): 
  
  message_lists = [[{"role": "user", 
                  "content": prompt},
                  ] for prompt in spanish_questions[i:i+batch_size]]

  # Transform the message list into a prompt string
  formatted_prompts = [llama_v2_prompt(message_list) for message_list in message_lists]
  
  with torch.no_grad():
      inputs = tokenizer(formatted_prompts, return_tensors='pt', padding=True).to('cuda')
      tokens = model.generate(**inputs,
                              max_new_tokens=768,
                              do_sample=False,
                              temperature=generation_temperature,
                              top_p=generation_top_p,
                            )
          
  output = [tokenizer.decode(seq, skip_special_tokens=True).split('[/INST]')[1] for seq in tokens]

  responses.extend(output)





['¿Puedes darme algunas sugerencias de atuendos? Voy a asistir a la fiesta de cumpleaños de mi amigo esta noche.', '¿Qué regalos de cumpleaños debo traer a mis amigos?']


100%|██████████| 1/1 [00:22<00:00, 22.38s/it]


In [21]:
for i in range(len(spanish_questions)):
    text = f"USER: {spanish_questions[i]}\n\n"
    text += "-" * 50 + "\n"
    text += f"Intervened:\n"
    text += f"CHATBOT: {responses[i]}"
    text += "\n\n" + "-" * 50 + "\n"
    
    print(text)

USER: ¿Puedes darme algunas sugerencias de atuendos? Voy a asistir a la fiesta de cumpleaños de mi amigo esta noche.

--------------------------------------------------
Intervened:
CHATBOT:   ¡Hola! ¡Claro que sí! Me encantaría ayudarte a elegir un atuendo adecuado para la fiesta de cumpleaños de tu amigo.

Para empezar, es importante tener en cuenta el tema de la fiesta y el ambiente en el que se celebrará. ¿Sabes si es una fiesta formal o informal? ¿Se celebrará en una discoteca, un parque o en una casa? Esto ayudará a determinar el tipo de atuendo que debes elegir.

Aquí te dejo algunas sugerencias de atuendos que podrían ser adecuados para una fiesta de cumpleaños:

1. Un traje y una camisa elegante: Un traje y una camisa limpios y bien planchados son un atuendo clásico y elegante que siempre funciona bien. Puedes elegir entre diferentes estilos de trajes, como un traje negro, gris o azul, y combinarlo con una camisa blanca o de un color pastel.
2. Un conjunto de pantalones y camis